## Cleaning

In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [4]:
%pip install nltk -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 55.5 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4
    Uninstalling regex-2021.4.4:
      Successfully uninstalled regex-2021.4.4
  Attempting uninstall: nltk
    Found existing installation: nltk 3.6.4
    Uninstalling nltk-3.6.4:
      Successfully uninstalled nltk-3.6.4
Note: you may need to restart the kernel to use updated packages.


In [5]:
import re
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import CountVectorizer,  IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [7]:
sc = spark.sparkContext
print('Original spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))

Original spark.driver.maxResultSize: 1920m


## Read the dataset

In [8]:
directory = 'gs://msca-bdp-tweets/final_project/'
# file = '*.json'
# path = directory + file
path = directory

In [9]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

498.7 G  498.7 G  gs://msca-bdp-tweets/final_project



In [10]:
%%time

tweets_df = spark.read.json(path)

23/02/26 20:23:24 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 1.79 s, sys: 354 ms, total: 2.15 s
Wall time: 7min 45s


23/02/26 20:28:51 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
%%time

tweets_df.count()

CPU times: user 467 ms, sys: 97.8 ms, total: 565 ms
Wall time: 2min 29s


99994342

In [11]:
tweets_df = tweets_df.filter(tweets_df.text.isNotNull())

In [13]:
tweets_df.limit(3)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries
null,Sun Oct 02 22:30:...,"[7, 47]","{[], null, [], []...",null,null,0,false,low,null,1576701112446750720,1576701112446750720,Dream,1576700948722110464,1576700948722110464,183905912,183905912,false,en,null,null,0,null,null,null,null,null,0,0,,null,null,"<a href=""http://t...",@Dream ARE YOU KI...,1664749813554,false,@Dream ARE YOU KI...,"{false, Sat Jan 0...",null
null,Sun Oct 02 22:30:...,"[13, 140]","{[{[75, 92], Shar...",null,"{[13, 259], {[{[7...",0,false,low,null,1576701112593571842,1576701112593571842,yoonieshope,1576649576504623104,1576649576504623104,1313841518638137349,1313841518638137349,false,en,null,null,0,null,null,null,null,null,0,0,,guardiannews @the...,null,"<a href=""http://t...",@yoonieshope @gua...,1664749813589,true,@yoonieshope @gua...,"{false, Fri Jul 2...",null
null,Sun Oct 02 22:30:...,null,"{[], null, [], []...",null,null,0,false,low,null,1576701113067524096,1576701113067524096,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,AviMayer,"{null, Thu Sep 29...","<a href=""http://t...",RT @AviMayer: .@U...,1664749813702,false,.@UCBerkeley is a...,"{false, Thu Apr 0...",null


In [ ]:
# https://dev.twitter.com/overview/api/tweets

In [14]:
sample = tweets_df.limit(10000).toPandas()

In [15]:
sample.shape

(10000, 39)

In [16]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   coordinates                4 non-null      object 
 1   created_at                 10000 non-null  object 
 2   display_text_range         1453 non-null   object 
 3   entities                   10000 non-null  object 
 4   extended_entities          376 non-null    object 
 5   extended_tweet             1605 non-null   object 
 6   favorite_count             10000 non-null  int64  
 7   favorited                  10000 non-null  bool   
 8   filter_level               10000 non-null  object 
 9   geo                        4 non-null      object 
 10  id                         10000 non-null  int64  
 11  id_str                     10000 non-null  object 
 12  in_reply_to_screen_name    1380 non-null   object 
 13  in_reply_to_status_id      1336 non-null   floa

In [17]:
sample.isnull().sum()

coordinates                   9996
created_at                       0
display_text_range            8547
entities                         0
extended_entities             9624
extended_tweet                8395
favorite_count                   0
favorited                        0
filter_level                     0
geo                           9996
id                               0
id_str                           0
in_reply_to_screen_name       8620
in_reply_to_status_id         8664
in_reply_to_status_id_str     8664
in_reply_to_user_id           8620
in_reply_to_user_id_str       8620
is_quote_status                  0
lang                             0
place                         9950
possibly_sensitive            8917
quote_count                      0
quoted_status                 9146
quoted_status_id              9144
quoted_status_id_str          9144
quoted_status_permalink       9146
quoted_text                   9146
reply_count                      0
retweet_count       

In [18]:
sample['created_at'] = pd.to_datetime(sample['created_at'])

# Create new columns for days of the week, month, and hour
sample['day_of_week'] = sample['created_at'].dt.day_name()
sample['month'] = sample['created_at'].dt.month_name()
sample['hour'] = sample['created_at'].dt.hour

In [24]:
sample['lang'].unique()

array(['en'], dtype=object)

In [62]:
sample[sample['retweeted'].notnull()]['retweeted'].unique()

array(['RT', ''], dtype=object)

In [34]:
#sample[sample['user'].notnull()].head(2)

,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,...,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries,day_of_week,month,hour
0,None,2022-05-25 21:22:46+00:00,None,"([], None, [], [], [(3103505578, 3103505578, [3, 15], Aaron Parnas, AaronParnas)])",None,None,0,False,low,None,...,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>","RT @AaronParnas: If you are more upset at Beto O'Rourke for interrupting a press conference than you are about the school shooting, you are…",1653513766765,False,"If you are more upset at Beto O'Rourke for interrupting a press conference than you are about the school shooting, you are the problem.","(False, Sat Nov 07 01:54:27 +0000 2015, True, False, I worked as CNA at a hospital for 10years and 2 Nursing homes , I worked for Endoscopy for a doctor , I am married and have three beautiful girls ., 12977, 274, 227, True, 4130836947, 4130836947, False, 2, Montgomery, AL, Lovenothate, C0DEED, http://abs.twimg.com/images/themes/theme1/bg.png, https://abs.twimg.com/images/themes/theme1/bg.png, False, https://pbs.twimg.com/profile_banners/4130836947/1614976685, http://pbs.twimg.com/profile_images/1367937055876120577/IXHi447P_normal.jpg, https://pbs.twimg.com/profile_images/1367937055876120577/IXHi447P_normal.jpg, 1DA1F2, C0DEED, DDEEF6, 333333, True, False, RamskiC, 10473, none, None, False, None, [])",None,Wednesday,May,21
1,None,2022-05-25 21:22:46+00:00,None,"([], None, [], [(twitter.com/i/web/status/1…, https://twitter.com/i/web/status/1529573716602064901, [117, 140], https://t.co/Jefm76MHMD)], [])",None,"([0, 228], ([], None, [], [], []), None, Fun fact: In Germany, homeschooling is effectively prohibited. You must send your child to an accredited school. Exceptions are made for ""the travelling people"" and a few others, but even there, the state decides the curriculum.)",0,False,low,None,...,"<a href=""https://ifttt.com"" rel=""nofollow"">IFTTT</a>","Fun fact: In Germany, homeschooling is effectively prohibited. You must send your child to an accredited school. Ex… https://t.co/Jefm76MHMD",1653513766800,True,"Fun fact: In Germany, homeschooling is effectively prohibited. You must send your child to an accredited school. Exceptions are made for ""the travelling people"" and a few others, but even there, the state decides the curriculum.","(False, Fri Jan 11 14:01:15 +0000 2008, False, False, Forest creature living in the plains. 💕 Empathy, Programming Languages, Drawing, Parking Cars. Streams Weekdays 7am CEST https://t.co/1iZKdRJkam. (Any pronouns), 28587, 2863, 979, False, 12114432, 12114432, False, 216, @uliwitness@mastodon.technology, Uli Kusterer, 1A1B1F, http://abs.twimg.com/images/themes/theme9/bg.gif, https://abs.twimg.com/images/themes/theme9/bg.gif, False, https://pbs.twimg.com/profile_banners/12114432/1355402696, http://pbs.twimg.com/profile_images/1422108028522881028/hQHtdjvS_normal.jpg, https://pbs.twimg.com/profile_images/1422108028522881028/hQHtdjvS_normal.jpg, 2FC2EF, 181A1E, 252429, 666666, True, False, uliwitness, 117472, none, http://orangejuiceliberationfront.com, False, None, [])",None,Wednesday,May,21


In [40]:
sample[sample['geo'].notnull()][['geo','place']]

,geo,place
957,"([44.0682019, -114.7420408], Point)","(([[[-117.243028, 41.987982], [-117.243028, 49.001121], [-111.043497, 49.001121], [-111.043497, 41.987982]]], Polygon), United States, US, Idaho, USA, 4723507d8ce23a60, Idaho, admin, https://api.twitter.com/1.1/geo/id/4723507d8ce23a60.json)"
1705,"([43.89755, -79.3038399], Point)","(([[[-79.429128, 43.798004], [-79.429128, 43.963385], [-79.170217, 43.963385], [-79.170217, 43.798004]]], Polygon), Canada, CA, Markham, Ontario, 7d2673c3623fe492, Markham, city, https://api.twitter.com/1.1/geo/id/7d2673c3623fe492.json)"
3892,"([41.8239891, -71.4128343], Point)","(([[[-71.474186, 41.772455], [-71.474186, 41.861713], [-71.369479, 41.861713], [-71.369479, 41.772455]]], Polygon), United States, US, Providence, RI, 7b93be1d864cedbb, Providence, city, https://api.twitter.com/1.1/geo/id/7b93be1d864cedbb.json)"
7207,"([22.80192986, 75.85570755], Point)","(([[[75.639557, 22.782867], [75.639557, 23.04095], [76.049073, 23.04095], [76.049073, 22.782867]]], Polygon), India, IN, Sanwer, India, 17be29db8450854c, Sanwer, city, https://api.twitter.com/1.1/geo/id/17be29db8450854c.json)"


In [55]:
# keep: [('day_of_week', 'month', 'hour'), 'favorite_count', 'filter_level', 'id_str', 'place', 
# 'quote_count', 'reply_count', 'retweeted', 'retweeted_from', 'source', 'text', 'user', 'timestamp_ms'


#Twitter measures the importance of a tweet using this parameter.It is a rating that Twitter gives to 
#its tweets. A tweet with filter_level=high is identified as more important (a part of @toptweets concept). 
#If you set filter_level=low, then you are asking for tweets with low, medium and high filter levels.
#You set the minimum quality of the tweets that you want using filter_level.

# in_reply_to_user_id_str: original user id
# in_reply_to_status_id_str: original tweet id
# is_quote_status: quote (yes/no)
# truncated: whether the tweet is truncated or not; true = is truncated = > 280 characters (contained in extended_tweet)

In [35]:
# user: 'id_str', 'name', 'screen_name', 'location', 'verified', 'followers_count'
# place: 'place_type', 'full_name', 'country'

In [12]:
select_df = tweets_df.select([tweets_df['created_at'],
                              tweets_df['id'].alias('unique_id'),
                              tweets_df['favorite_count'],
                              tweets_df['place']['place_type'].alias('tweet_location_type'),
                              tweets_df['place']['full_name'].alias('tweet_location'),
                              tweets_df['place']['country'].alias('tweet_country'),
                              tweets_df['quote_count'],
                              tweets_df['reply_count'],
                              tweets_df['retweeted_status']['retweet_count'].alias('retweet_count'),
                              tweets_df['retweeted'],
                              tweets_df['retweeted_from'],
                              tweets_df['source'],
                              tweets_df['tweet_text'],
                              tweets_df['timestamp_ms'],
                              tweets_df['id_str'].alias('user_id'),
                              tweets_df['user']['name'].alias('user_name'),
                              tweets_df['user']['screen_name'].alias('user_screen_name'),
                              tweets_df['user']['description'].alias('user_description'),
                              tweets_df['user']['location'].alias('user_location'),
                              tweets_df['user']['verified'].alias('user_verified'),
                              tweets_df['user']['followers_count'].alias('followers_count')])

In [23]:
select_df.limit(3)

created_at,favorite_count,tweet_location_type,tweet_location,tweet_country,quote_count,reply_count,retweeted,retweeted_from,source,tweet_text,timestamp_ms,user_id,user_name,user_screen_name,user_location,user_verified,followers_count
Wed May 25 21:22:...,0,null,null,null,0,0,RT,AaronParnas,"<a href=""http://t...",If you are more u...,1653513766765,1529573716455202817,Lovenothate,RamskiC,"Montgomery, AL",false,274
Wed May 25 21:22:...,0,null,null,null,0,0,,null,"<a href=""https://...",Fun fact: In Germ...,1653513766800,1529573716602064901,Uli Kusterer,uliwitness,@uliwitness@masto...,false,2863
Wed May 25 21:22:...,0,null,null,null,0,0,RT,MoneyMiaaaa,"<a href=""https://...",26 school shootin...,1653513766876,1529573716921040896,Ryott,Ryott13,null,false,5


In [ ]:
select_df.count()

99994342

In [13]:
select_df = select_df.withColumn('tweet_text', lower(col('tweet_text')))

In [17]:
text = select_df.rdd.map(lambda x : x['tweet_text']).filter(lambda x: x is not None)

StopWords = stopwords.words("english")

tokens = text\
            .map( lambda document: document.strip().lower())\
            .map( lambda document: re.split(" ", document))\
            .map( lambda word: [x for x in word if x.isalnum()])\
            .map( lambda word: [x for x in word if x not in StopWords])\
            .map( lambda word: [x for x in word if len(x) > 3] ) \
            .map( lambda word: ' '.join(word))

In [18]:
tokens.take(5)

['upset beto interrupting press conference school',
 'homeschooling effectively must send child accredited exceptions made travelling even state decides',
 'school shootings worried ridiculous',
 'school shootings worried ridiculous',
 'uncompleted structure fenced gated adebisi worship residential mixed']

In [19]:
%%time

wordCounts = tokens.flatMap(lambda text: text.split(' ')) \
                    .map(lambda word: (word, 1)) \
                    .reduceByKey(lambda a, b: a+b)


wordCountsSorted = wordCounts.map(lambda x:(x[1],x[0])).sortByKey(ascending=False)

CPU times: user 4.04 s, sys: 1.58 s, total: 5.62 s
Wall time: 12min 3s


In [20]:
wordCountsSorted.take(60)

[(46730255, 'school'),
 (13142300, 'college'),
 (10485482, 'university'),
 (9941284, 'high'),
 (8489925, 'schools'),
 (6424243, 'students'),
 (6190310, 'like'),
 (4900205, 'kids'),
 (4816268, 'people'),
 (3859045, 'children'),
 (3626777, 'would'),
 (3553623, 'public'),
 (3550440, 'professor'),
 (3423061, 'back'),
 (3310736, 'student'),
 (3307705, 'time'),
 (3190282, 'first'),
 (3042286, 'know'),
 (3034183, 'need'),
 (3012764, 'year'),
 (3012160, 'want'),
 (2796166, 'going'),
 (2757656, 'make'),
 (2723719, 'state'),
 (2702628, 'years'),
 (2638131, 'even'),
 (2551203, 'education'),
 (2547529, 'think'),
 (2541635, 'help'),
 (2516575, 'every'),
 (2391271, 'still'),
 (2353474, 'football'),
 (2324686, 'never'),
 (2310563, 'went'),
 (2277740, 'today'),
 (2213762, 'good'),
 (2195981, 'also'),
 (2143748, 'many'),
 (2139303, 'take'),
 (2135759, 'parents'),
 (2088702, 'teacher'),
 (2057427, 'work'),
 (2042095, 'teachers'),
 (2023053, 'home'),
 (2021871, 'free'),
 (2010681, 'live'),
 (1968589, 'sa

In [14]:
keywords = ['school', 'college', 'university', 'schools', 'students', 'kids', 
            'professor', 'student', 'children', 'public', 'education', 'parents',
            'teacher', 'teachers', 'class', 'elementary', 'high', 'middle', 'professors',
            'private', 'football', 'state']

#clean_df = select_df.filter(select_df.tweet_text.contain(keywords))

In [15]:
filter_expr = select_df.tweet_text.contains(keywords[0])

for keyword in keywords[1:]:
    filter_expr |= select_df.tweet_text.contains(keyword)

clean_df = select_df.filter(filter_expr)

In [24]:
clean_df.limit(5)

created_at,favorite_count,tweet_location_type,tweet_location,tweet_country,quote_count,reply_count,retweeted,retweeted_from,source,tweet_text,timestamp_ms,user_id,user_name,user_screen_name,user_location,user_verified,followers_count
Wed May 25 03:41:...,0,null,null,null,0,0,RT,ArtValley818_,"<a href=""http://t...",instead of sendin...,1653450098896,1529306674057601024,Clifford Bauman Sr.,CliffordBauman,"Dale City, VA",false,312
Wed May 25 03:41:...,0,null,null,null,0,0,RT,romyreiner,"<a href=""http://t...",you can kill chil...,1653450098997,1529306674481074176,Fernando Sabines,FernandoSabines,"San Francisco, C...",false,1718
Wed May 25 03:41:...,0,null,null,null,0,0,RT,BarackObama,"<a href=""http://t...",across the countr...,1653450099017,1529306674565201921,•,Fangurl2019,Heart is at Wrigley,false,514
Wed May 25 03:41:...,0,null,null,null,0,0,,josh_chagnon @Col...,"<a href=""http://t...",@yvonnef85886180 ...,1653450099017,1529306674565107714,CattyPatty,cattypatty22,null,false,509
Wed May 25 03:41:...,0,null,null,null,0,0,,null,"<a href=""http://t...",@logantillman mat...,1653450099019,1529306674573590528,Matthew Boylan 6’...,matthewboylan13,"Boca Raton, FL",false,110


In [16]:
clean_df.write.format('parquet').\
mode('overwrite').\
save('gs://msca-bdp-students-bucket/shared_data/qsliu/clean_df')